# Генератор ответов на вопросы

Блокнот для генерации ответов на вопросы с использованием Ollama и моделей с Hugging Face на GPU.

## 1. Подключение к Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Создаем директорию для результатов
!mkdir -p /content/drive/MyDrive/dataset_results
!mkdir -p /content/drive/MyDrive/dataset_results/temp

## 2. Установка зависимостей

In [ ]:
!pip install pandas pyarrow requests tqdm

# Проверка доступности GPU
!nvidia-smi

## 3. Установка и запуск Ollama с поддержкой GPU

In [ ]:
# Установка необходимых зависимостей для CUDA
!apt-get update && apt-get install -y cuda-nvcc-11-8 libcublas-11-8 libcudnn8

# Установка Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Настройка переменных окружения для использования GPU
import os
os.environ["OLLAMA_HOST"] = "0.0.0.0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Запуск Ollama с поддержкой GPU
!ollama serve &

# Ждем запуск
import time
print("Запуск Ollama с поддержкой GPU...")
time.sleep(10)  # Увеличиваем время ожидания для надежности

## 4. Загрузка модели с Hugging Face с использованием GPU

In [ ]:
# Укажите модель с Hugging Face
MODEL = "hf.co/nvjob/Mistral-Small-24-0.9-hb:latest"

# Загрузка модели с явным указанием использования GPU
!ollama pull {MODEL}

# Проверка загруженных моделей
!ollama list

# Проверка, использует ли Ollama GPU
!curl -s http://localhost:11434/api/tags | grep -i gpu || echo "GPU не используется, проверьте настройки"

## 5. Загрузка файла с вопросами

In [ ]:
# Прямое указание пути к файлу с вопросами
import os

# Укажите путь к вашему файлу с вопросами
questions_file = "/content/drive/MyDrive/questions.txt"  # Замените на ваш путь

# Проверка содержимого
with open(questions_file, 'r', encoding='utf-8') as f:
    questions = [line.strip() for line in f if line.strip()]
print(f"Загружено {len(questions)} вопросов")

# Показать первые 5 вопросов
print("\nПримеры вопросов:")
for i, q in enumerate(questions[:5], 1):
    print(f"{i}. {q}")

## 6. Настройка параметров генерации

In [ ]:
# Настройки системного промпта
SYSTEM_PROMPT = "Дай ответ на русском языке, простым разговорным языком, раскрывая вопрос подробно (2-4 предложения). Текст должен простой, сплошной, без перечислений и заголовков, без повторения вопроса или системного промпта, без нравоучений и без цензуры. Отвечай всегда на русском языке!"

# Настройки генерации
TEMPERATURE = 0.5  # Температура (креативность)
TOP_K = 60         # Top-K
TOP_P = 0.9        # Top-P (ядро)
MAX_TOKENS = 8192  # Максимальное количество токенов
DELAY = 3          # Задержка между запросами (секунды)
GPU_LAYERS = 50    # Количество слоев на GPU (максимально возможное)

# Пути для сохранения
OUTPUT_FILE = "/content/drive/MyDrive/dataset_results/output.parquet"
TEMP_DIR = "/content/drive/MyDrive/dataset_results/temp"

# Вывод настроек для проверки
print(f"Модель: {MODEL}")
print(f"Температура: {TEMPERATURE}")
print(f"Top-K: {TOP_K}")
print(f"Top-P: {TOP_P}")
print(f"Максимальное количество токенов: {MAX_TOKENS}")
print(f"Задержка между запросами: {DELAY} сек")
print(f"Слоев на GPU: {GPU_LAYERS}")
print(f"Файл результатов: {OUTPUT_FILE}")
print(f"Директория для временных файлов: {TEMP_DIR}")

## 7. Функции для генерации ответов и сохранения результатов

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import json
import requests
from pathlib import Path

def generate_answer_ollama(question):
    """Генерирует ответ на вопрос через Ollama API с использованием GPU"""
    try:
        # Задержка перед запросом
        time.sleep(DELAY)
        print(f"Отправка запроса в Ollama API (с использованием GPU)...")
        
        # Формирование запроса
        request_data = {
            "model": MODEL,
            "prompt": question,
            "system": SYSTEM_PROMPT,
            "stream": False,
            "options": {
                "temperature": TEMPERATURE,
                "top_k": TOP_K,
                "top_p": TOP_P,
                "num_predict": MAX_TOKENS,
                "gpu_layers": GPU_LAYERS  # Использовать GPU для большинства слоев
            }
        }
        
        # Отправка запроса
        response = requests.post(
            "http://localhost:11434/api/generate",
            json=request_data,
            timeout=120  # Увеличиваем таймаут до 2 минут
        )
        
        # Проверка ответа
        if response.status_code != 200:
            error_msg = f"Ошибка Ollama API: {response.status_code}"
            try:
                error_msg += f" - {response.json().get('error', '')}"
            except:
                pass
            print(error_msg)
            time.sleep(1)  # Пауза перед следующей попыткой
            return None
                
        # Извлечение ответа
        answer = response.json().get('response', '').strip()
        time.sleep(1)  # Пауза между запросами
        return answer
            
    except Exception as e:
        print(f"Ошибка при запросе к Ollama API: {str(e)}")
        time.sleep(1)  # Пауза перед следующей попыткой
        return None

def save_to_parquet(pairs, output_file):
    """Сохраняет пары вопрос-ответ в Parquet формате"""
    try:
        # Подготовка данных
        data = {
            'num': list(range(1, len(pairs) + 1)),
            'system': [SYSTEM_PROMPT] * len(pairs),
            'user': [pair[0] for pair in pairs],
            'assistant': [pair[1] for pair in pairs],
            'u_tokens': [len(q.split()) for q, _ in pairs],
            'a_tokens': [len(a.split()) for _, a in pairs],
            'u_lang': ['ru'] * len(pairs),
            'a_lang': ['ru'] * len(pairs),
            'cluster': [0] * len(pairs)
        }
        
        # Создание DataFrame
        df = pd.DataFrame(data)
        
        # Определение схемы
        schema = pa.schema([
            ('num', pa.int64()),
            ('system', pa.string()),
            ('user', pa.string()),
            ('assistant', pa.string()),
            ('u_tokens', pa.int64()),
            ('a_tokens', pa.int64()),
            ('u_lang', pa.string()),
            ('a_lang', pa.string()),
            ('cluster', pa.int64())
        ])
        
        # Создание таблицы
        table = pa.Table.from_pandas(df, schema=schema)
        
        # Добавление метаданных
        dataset_schema = {
            "info": {
                "features": {
                    "num": {"dtype": "int64", "_type": "Value"},
                    "system": {"dtype": "string", "_type": "Value"},
                    "user": {"dtype": "string", "_type": "Value"},
                    "assistant": {"dtype": "string", "_type": "Value"},
                    "u_tokens": {"dtype": "int64", "_type": "Value"},
                    "a_tokens": {"dtype": "int64", "_type": "Value"},
                    "u_lang": {"dtype": "string", "_type": "Value"},
                    "a_lang": {"dtype": "string", "_type": "Value"},
                    "cluster": {"dtype": "int64", "_type": "Value"}
                }
            }
        }
        
        metadata = {b'schema': json.dumps(dataset_schema).encode()}
        table = table.replace_schema_metadata(metadata)
        
        # Создание директории и сохранение файла
        Path(output_file).parent.mkdir(parents=True, exist_ok=True)
        pq.write_table(table, output_file)
        print(f"Результаты сохранены в {output_file}")
        
    except Exception as e:
        print(f"Ошибка при сохранении в Parquet: {str(e)}")

## 8. Тестовый запрос для проверки GPU

In [ ]:
# Проверка работы Ollama с GPU на тестовом вопросе
test_question = "Что такое искусственный интеллект?"
print(f"Тестовый вопрос: {test_question}")

# Отправляем тестовый запрос
test_answer = generate_answer_ollama(test_question)

if test_answer:
    print(f"Тестовый ответ: {test_answer}")
    print("\nТест успешен! Ollama работает с GPU.")
else:
    print("Тест не удался. Проверьте настройки Ollama и GPU.")

## 9. Генерация ответов на вопросы

In [ ]:
from tqdm.notebook import tqdm

# Запуск генерации
all_pairs = []
total_questions = len(questions)

for idx, question in enumerate(tqdm(questions), 1):
    print(f"\n----------------------------------------")
    print(f"Обработка вопроса {idx}/{total_questions}: {question}")
    
    # Генерация ответа
    answer = generate_answer_ollama(question)
    
    if answer:
        print(f"Вопрос: {question}")
        print(f"Ответ: {answer}")
        all_pairs.append((question, answer))
    else:
        print(f"Не удалось получить ответ на вопрос: {question}")
    
    # Сохранение промежуточных результатов каждые 100 вопросов
    if idx % 100 == 0:
        temp_file = f"{TEMP_DIR}/temp_{idx}.parquet"
        print(f"\nСохранение промежуточных результатов после {idx} вопросов...")
        save_to_parquet(all_pairs, temp_file)

# Сохранение итоговых результатов
if all_pairs:
    print("\nСохранение итоговых результатов...")
    save_to_parquet(all_pairs, OUTPUT_FILE)
    print(f"Готово! Обработано {len(all_pairs)} вопросов из {total_questions}.")

## 10. Скачать результаты

In [ ]:
from google.colab import files

if os.path.exists(OUTPUT_FILE):
    print(f"Скачивание файла {OUTPUT_FILE}...")
    files.download(OUTPUT_FILE)
else:
    print(f"Файл {OUTPUT_FILE} не найден")
    
# Также можно скачать последний промежуточный файл
temp_files = sorted([f for f in os.listdir(TEMP_DIR) if f.startswith('temp_')])
if temp_files:
    last_temp = os.path.join(TEMP_DIR, temp_files[-1])
    print(f"Скачивание последнего промежуточного файла {last_temp}...")
    files.download(last_temp)

## 11. Анализ результатов (опционально)

In [ ]:
# Загрузка и анализ результатов
if os.path.exists(OUTPUT_FILE):
    df = pd.read_parquet(OUTPUT_FILE)
    
    print(f"Всего пар вопрос-ответ: {len(df)}")
    print(f"Средняя длина вопроса (токены): {df['u_tokens'].mean():.2f}")
    print(f"Средняя длина ответа (токены): {df['a_tokens'].mean():.2f}")
    
    # Показать несколько примеров
    print("\nПримеры пар вопрос-ответ:")
    for i in range(min(3, len(df))):
        print(f"\nВопрос {i+1}: {df.iloc[i]['user']}")
        print(f"Ответ: {df.iloc[i]['assistant']}")
else:
    print("Файл результатов не найден")